In [1]:
import cv2
import os
import numpy as np
from keras.utils import np_utils

In [2]:
data_path=('E:/opencv/data')
categories=os.listdir(data_path)
labels=[i for i in range(0,len(categories))]

label_dict=dict(zip(categories,labels))

In [3]:
img_size=100
data=[]
target=[]

for cat in categories:
    folder_path=os.path.join(data_path,cat)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        
        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[cat])
            
        except Exception as e:
            print('Invalid image file!\n',e)

In [4]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))

target=np.array(target)
target=np_utils.to_categorical(target)

In [5]:
#np.save('imgdata',data)
#np.save('imgtarget',target)

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [7]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2)

In [9]:
checkpoint=ModelCheckpoint('mask-{epoch:02d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

history=model.fit(X_train,y_train,epochs=20,callbacks=[checkpoint],validation_data=(X_test,y_test))

Epoch 1/20
35/35 [==============================] - ETA: 0s - loss: 0.6959 - accuracy: 0.5309WARNING:tensorflow:From c:\users\decay\anaconda3\envs\opencv-env\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mask-01.model\assets
35/35 [==============================] - 36s 1s/step - loss: 0.6959 - accuracy: 0.5309 - val_loss: 0.6436 - val_accuracy: 0.6884
Epoch 2/20
35/35 [==============================] - 36s 1s/step - loss: 0.5744 - accuracy: 0.7082 - val_loss: 0.4497 - val_accuracy: 0.7899
Epoch 3/20
35/35 [==============================] - 36s 1s/step - loss: 0.4108 - accuracy: 0.8291 - val_loss: 0.3278 - val_accuracy: 0.8551
Epoch 4/20
35/35 [==============================] - 36s 1s/step -